# Final Presentation Demo team kA

This shows step by step the abilities of our package


In [125]:
import pandas as pd
import numpy as np
import pytest
from power_grid_model import ComponentType

from power_system_simulation.exceptions import *
from power_system_simulation.input_data_validation import *
from power_system_simulation.graph_processing import *
from power_system_simulation.power_grid_calculation import *

path_small_test_grid = "graph_cycle_disabled.json"

path_small_grid = "small_power_grid/input_network_data.json"
path_small_meta_data = "small_power_grid/meta_data.json"
path_small_p = "small_power_grid/active_power_profile.parquet"
path_small_q = "small_power_grid/reactive_power_profile.parquet"
path_small_ev = "small_power_grid/ev_active_power_profile.parquet"

path_large_grid = "large_power_grid/input_network_data.json"
path_large_meta_data = "large_power_grid/meta_data.json"
path_large_p = "large_power_grid/active_power_profile.parquet"
path_large_q = "large_power_grid/reactive_power_profile.parquet"
path_large_ev = "large_power_grid/ev_active_power_profile.parquet"


### Assignment 1: Graph processor

First, import a power_grid and create a graph out of it

In [126]:
power_grid = load_grid_json(path_small_test_grid)
graph = create_graph(power_grid)

In [127]:
"""
The small test grid layout:

1--<9>--2--[10]-3------(16)
^       |       .
20     [11]    [21]
        |       .
        4--[12]-5------(17)
        |       .
        [13]   [22]
        |       .
        6--[14]-7--[15]-8-----(19)
                |
               (18)

"""

'\nThe small test grid layout:\n\n1--<9>--2--[10]-3------(16)\n^       |       .\n20     [11]    [21]\n        |       .\n        4--[12]-5------(17)\n        |       .\n        [13]   [22]\n        |       .\n        6--[14]-7--[15]-8-----(19)\n                |\n               (18)\n\n'

### With this we can analyze some specifications and properties of the power-grid

Such as alternative lines when a specified line gets disabled.

In [ ]:
find_alternative_edges(graph, 11)

[np.int32(21)]


In [129]:
find_alternative_edges(graph, 12)

[np.int32(21), np.int32(22)]

In [130]:
find_alternative_edges(graph, 15)

[]

And also any downstream vertices with respect to a source node.

In [131]:
find_downstream_vertices(graph, 10)

[np.int32(3)]

In [132]:
find_downstream_vertices(graph, 11)

[np.int32(4), np.int32(5), np.int32(6), np.int32(7), np.int32(8)]

In [133]:
graph.graph["source_node_id"] = 4

find_downstream_vertices(graph, 11)

[np.int32(1), np.int32(2), np.int32(3)]

### Assignment 2: Power Flow simulations

This is accomplished by using the PowerGrid class. This class contains the logic for running the model, and generates a graph internally for easy tracking and updating of the graph with respect to the power-grid data.

In [134]:
power_grid = PowerGrid(path_small_grid, path_small_meta_data, path_small_p, path_small_q)
power_grid.run()

power_grid.line_summary

,energy_loss,max_loading_timestamp,max_loading,min_loading_timestamp,min_loading
line,,,,,
16,22.898797,2025-01-04 06:30:00,0.308152,2025-01-08 12:30:00,0.056235
17,1.172405,2025-01-04 09:45:00,0.160315,2025-01-08 11:30:00,0.026153
18,34.828580,2025-01-07 10:45:00,0.163169,2025-01-05 17:45:00,0.026844
19,1.333249,2025-01-07 10:45:00,0.163366,2025-01-05 17:45:00,0.026425
20,22.541055,2025-01-07 10:45:00,0.324606,2025-01-02 14:30:00,0.053688
21,1.220496,2025-01-07 10:30:00,0.161755,2025-01-02 14:30:00,0.024864
22,31.185873,2025-01-07 10:45:00,0.173607,2025-01-02 12:30:00,0.025599
23,1.394214,2025-01-07 10:45:00,0.173514,2025-01-02 12:30:00,0.025472
24,227.112446,2025-01-10 21:15:00,0.001279,2025-01-04 09:45:00,0.001262


In [135]:
power_grid.voltage_summary

,max_u_pu_node,max_u_pu,min_u_pu_node,min_u_pu
timestamp,,,,
2025-01-01 00:00:00,1,1.072931,0,1.049819
2025-01-01 00:15:00,1,1.075911,0,1.050022
2025-01-01 00:30:00,1,1.069725,0,1.049603
2025-01-01 00:45:00,1,1.073244,0,1.049842
2025-01-01 01:00:00,1,1.072924,0,1.049819
...,...,...,...,...
2025-01-10 22:45:00,1,1.071457,0,1.049730
2025-01-10 23:00:00,1,1.075341,0,1.049993
2025-01-10 23:15:00,1,1.072623,0,1.049803


In [136]:
"""
The small grid layout:

                    3------(12)
                    |
                   [17]
                    |    
           /--[16]--2--[18]--4--[19]--5------(13)
          /                  .
0--<11>--1                  [24]
^         \                  .
10         \--[20]--6--[22]--8--[23]--9------(15)
                    |
                   [21]
                    |
                    7------(14)

"""

'\nThe small grid layout:\n\n                    3------(12)\n                    |\n                   [17]\n                    |    \n           /--[16]--2--[18]--4--[19]--5------(13)\n          /                  .\n0--<11>--1                  [24]\n^         \\                  .\n10         \\--[20]--6--[22]--8--[23]--9------(15)\n                    |\n                   [21]\n                    |\n                    7------(14)\n\n'

In [138]:
find_alternative_edges(power_grid.graph, 22)

[np.int32(24)]

In [139]:
find_alternative_edges(power_grid.graph, 11)

[]

In [140]:
"""
The small grid layout:

                    3------(12)
                    |
                   [17]
                    |    
           /--[16]--2--[18]--4--[19]--5------(13)
          /                  .
0--<11>--1                  [24]
^         \                  .
10         \--[20]--6--[22]--8--[23]--9------(15)
                    |
                   [21]
                    |
                    7------(14)

"""

'\nThe small grid layout:\n\n                    3------(12)\n                    |\n                   [17]\n                    |    \n           /--[16]--2--[18]--4--[19]--5------(13)\n          /                  .\n0--<11>--1                  [24]\n^         \\                  .\n10         \\--[20]--6--[22]--8--[23]--9------(15)\n                    |\n                   [21]\n                    |\n                    7------(14)\n\n'

In [143]:
find_downstream_vertices(power_grid.graph, 11)

[np.int32(1),
 np.int32(2),
 np.int32(3),
 np.int32(4),
 np.int32(5),
 np.int32(6),
 np.int32(7),
 np.int32(8),
 np.int32(9)]

In [142]:
find_downstream_vertices(power_grid.graph, 16)

[np.int32(2), np.int32(3), np.int32(4), np.int32(5)]

In [ ]:
power_grid.graph.graph["source_node_id"] = 9

find_downstream_vertices(power_grid.graph, 20)